Este Notebook es para realizar pruebas de la estructura que tengo en mente.


In [ ]:
from Structure import Core
from nltk.corpus import stopwords
import itertools
import numpy as np
import pandas as pd

if __name__ == '__main__':
    
    # Lista de instrumentos que se van a analizar
    nombres_inst=['MATISSE','PIONIER','GRAVITY']
    n= len(nombres_inst) # Número de instrumentos
    BASES={}  # Diccionario para almacenar instancias de la clase Core
    
    # Iterar sobre cada instrumento
    for nombre_instrumento in nombres_inst:

        # Se crea una instancia de la clase Core, se puede modificar el rango de tiempo
        Base = Core(INSTRUMENT=nombre_instrumento ,RANGE='6m',time_it=True)
        BASES[nombre_instrumento] = Base  # Almacenar Core en el diccionario
        
        # Número de errores en el conjunto de metadatos del instrumento actual
        n_errores = BASES[nombre_instrumento].n_errors
        BASES[nombre_instrumento].meta_tokenization()  # Tokenizar los metadatos
        stop_words = stopwords.words('english')  # Obtener la lista de stopwords en inglés
        BASES[nombre_instrumento].meta_vectorization(stop_words = stop_words)  # Vectorizar los metadatos utilizando las stopwords


        # Crear un rango de valores para el parámetro alpha y para fit_prior
        alpha_range = np.arange(0,1,0.1).tolist()
        fit_prior = [True, False]
        sampled_success = range(n_errores, n_errores*7 + 1, n_errores)
        combinaciones = list(itertools.product(alpha_range,fit_prior,sampled_success))  # Crear combinaciones de parámetros

        regexp_list = [r'\D+', r'[a-zA-Z]+', r'\s+', r'\b[a-zA-Z]+\b'] # Se testearán 4 expresiones regulares
        for regexp in regexp_list:
            if regexp != r'\s+': BASES[nombre_instrumento].meta_tokenization(regexp=regexp)
            else: BASES[nombre_instrumento].meta_tokenization(regexp=regexp, gaps=True)
            for combinacion in combinaciones:  # Iterar sobre cada combinación de parámetros
                alpha, fit_prior, sampled_success = combinacion

                # Crear un diccionario con los parámetros para el clasificador Naive Bayes
                NB_args = {'alpha': alpha,
                        'fit_prior': fit_prior}
                
                # Llamar al pipeline de clasificación usando Naive Bayes
                BASES[nombre_instrumento].call_pipeline(method = 'Multinomial_NB', sample_success= sampled_success, NB_kwargs = NB_args)

        # Definir un rango de umbrales para el análisis de sentimientos
        tresholds = np.arange(0.5,1.5,0.1).tolist()

        # Iterar sobre cada umbral y ejecutar el análisis de sentimientos
        for regexp in regexp_list:
            if regexp != r'\s+': BASES[nombre_instrumento].meta_tokenization(regexp=regexp)
            else: BASES[nombre_instrumento].meta_tokenization(regexp=regexp, gaps=True)
            for treshold in tresholds:
                BASES[nombre_instrumento].call_pipeline(method = 'TextBlob', treshold = treshold)

        # Crear un DataFrame resumen con los resultados del pipeline
        summary = pd.DataFrame(BASES[nombre_instrumento].pipeline.data)

        # Guardar el resumen en un archivo CSV
        summary.to_csv(f'{nombre_instrumento}.csv')

Loading Metadata for: 
INSTRUMENT: MATISSE 
RANGE: 6m 

Done.


In [3]:
from Structure import Core

#Base = Core(RANGE='6m')
Base.n_errors

270

In [5]:
print(Base.dataset.info())
print(Base.meta_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22497543 entries, 0 to 22497542
Data columns (total 14 columns):
 #   Column      Dtype         
---  ------      -----         
 0   @timestamp  datetime64[ns]
 1   system      object        
 2   hostname    object        
 3   loghost     object        
 4   logtype     object        
 5   envname     object        
 6   procname    object        
 7   procid      float64       
 8   module      object        
 9   keywname    object        
 10  keywvalue   object        
 11  keywmask    object        
 12  logtext     object        
 13  trace_id    int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(11)
memory usage: 2.3+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6653 entries, 0 to 6652
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   START     6653 non-null   datetime64[ns]
 1   END       6653 non-null   datetime64

In [3]:
a,b,c = combinaciones[0]

In [ ]:
for combinacion in combinaciones:
    alpha,fit_prior,sampled_success = combinacion
    NB_args = {'alpha': alpha,
               'fit_prior': fit_prior
               }
    Base.call_pipeline(method='Multinomial_NB', sample_success=sampled_success,NB_kwargs=NB_args)

In [13]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
summary = pd.DataFrame(Base.pipeline.data)

In [15]:
summary.sort_values(by='Precision',ascending=False).head(30)

,Classifier,Success_Sampling,Params,True_negative,False_negative,True_positive,False_positive,Accuracy,Precision,Recall
135,"MultinomialNB(alpha=0.7000000000000001, fit_pr...",100,"{'alpha': 0.7000000000000001, 'fit_prior': False}",12,1,8,8,0.689655,0.500000,0.888889
144,MultinomialNB(alpha=0.8),100,"{'alpha': 0.8, 'fit_prior': True}",12,1,8,8,0.689655,0.500000,0.888889
90,MultinomialNB(alpha=0.5),100,"{'alpha': 0.5, 'fit_prior': True}",12,2,7,8,0.655172,0.466667,0.777778
81,"MultinomialNB(alpha=0.4, fit_prior=False)",100,"{'alpha': 0.4, 'fit_prior': False}",10,1,8,10,0.620690,0.444444,0.888889
73,MultinomialNB(alpha=0.4),200,"{'alpha': 0.4, 'fit_prior': True}",31,2,7,9,0.775510,0.437500,0.777778
37,MultinomialNB(alpha=0.2),200,"{'alpha': 0.2, 'fit_prior': True}",31,2,7,9,0.775510,0.437500,0.777778
18,MultinomialNB(alpha=0.1),100,"{'alpha': 0.1, 'fit_prior': True}",11,2,7,9,0.620690,0.437500,0.777778
163,MultinomialNB(alpha=0.9),200,"{'alpha': 0.9, 'fit_prior': True}",30,2,7,10,0.755102,0.411765,0.777778
72,MultinomialNB(alpha=0.4),100,"{'alpha': 0.4, 'fit_prior': True}",10,2,7,10,0.586207,0.411765,0.777778
153,"MultinomialNB(alpha=0.8, fit_prior=False)",100,"{'alpha': 0.8, 'fit_prior': False}",10,2,7,10,0.586207,0.411765,0.777778


In [6]:
Base.trace_tokenization()  # el tokenizador de trazas puede recibir argumetos en expresiones regulares para probar distíntos filtros

Execution time:  0.024202400003559887  Seconds


60067             [Celestial, target, acquisition, yellow]
60068                            [Started, at, underlined]
60069                                      [WLMINCUT, VAL]
60070                         [COU, AG, GSSOURCE, SCIENCE]
60071                                     [COU, AG, EPOCH]
                               ...                        
62950                               [SETUP, command, done]
62951                                                   []
62952                                 [Template, finished]
62953              [Finished, in, seconds, at, underlined]
62954    [TplExecTimeStats, TPL, ID, estimated, real, d...
Name: logtext, Length: 2888, dtype: object

In [7]:
Base.trace_tokenization(regexp=r'\D+')

Execution time:  0.0172131999861449  Seconds


60067    [MATISSE_img_acq -- Celestial target acquisiti...
60068          [Started at , -, -, T, :, :,  (underlined)]
60069                       [DET,  WLMINCUT VAL = ', ., ']
60070                        [COU AG GSSOURCE = 'SCIENCE']
60071                                [COU AG EPOCH = ', ']
                               ...                        
62950                                [SETUP command done.]
62951                                [-,  (SpringGreen, )]
62952                 [Template MATISSE_img_acq finished.]
62953    [Finished in ,  seconds at , -, -, T, :, :,  (...
62954    [TplExecTimeStats: TPL.ID MATISSE_img_acq  est...
Name: logtext, Length: 2888, dtype: object

In [11]:
Base.trace_tokenization(regexp=r'[a-zA-Z]+')

Execution time:  0.02167299995198846  Seconds


60067    [MATISSE, img, acq, Celestial, target, acquisi...
60068                         [Started, at, T, underlined]
60069                                 [DET, WLMINCUT, VAL]
60070                         [COU, AG, GSSOURCE, SCIENCE]
60071                                     [COU, AG, EPOCH]
                               ...                        
62950                               [SETUP, command, done]
62951                                        [SpringGreen]
62952              [Template, MATISSE, img, acq, finished]
62953           [Finished, in, seconds, at, T, underlined]
62954    [TplExecTimeStats, TPL, ID, MATISSE, img, acq,...
Name: logtext, Length: 2888, dtype: object

In [9]:
Base.trace_tokenization(regexp=r'\s+', gaps=True)

Execution time:  0.02972099999897182  Seconds


60067    [MATISSE_img_acq, --, Celestial, target, acqui...
60068     [Started, at, 2019-04-10T02:29:03, (underlined)]
60069                      [DET1, WLMINCUT, VAL, =, '3.0']
60070                    [COU, AG, GSSOURCE, =, 'SCIENCE']
60071                          [COU, AG, EPOCH, =, '2000']
                               ...                        
62950                              [SETUP, command, done.]
62951                                 [-1, (SpringGreen4)]
62952               [Template, MATISSE_img_acq, finished.]
62953    [Finished, in, 334, seconds, at, 2019-04-10T02...
62954    [TplExecTimeStats:, TPL.ID, MATISSE_img_acq, e...
Name: logtext, Length: 2888, dtype: object